In [2]:
import tensorflow as tf
import foolbox as fb

model = tf.keras.applications.ResNet50(weights="imagenet")
preprocessing = dict(flip_axis=-1, mean=[103.939, 116.779, 123.68])
bounds = (0, 255)
fmodel = fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [3]:
fmodel = fmodel.transform_bounds((0, 1))

In [4]:
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=16)

In [12]:
images

<tf.Tensor: shape=(16, 224, 224, 3), dtype=float32, numpy=
array([[[[0.7490196 , 0.6784314 , 0.53333336],
         [0.75686276, 0.6901961 , 0.5137255 ],
         [0.8156863 , 0.7411765 , 0.5803922 ],
         ...,
         [0.21960784, 0.03529412, 0.07058824],
         [0.22745098, 0.03529412, 0.07058824],
         [0.23137255, 0.0627451 , 0.07450981]],

        [[0.76862746, 0.69803923, 0.5411765 ],
         [0.7411765 , 0.67058825, 0.48235294],
         [0.7647059 , 0.6901961 , 0.5058824 ],
         ...,
         [0.24313726, 0.01960784, 0.07843138],
         [0.22745098, 0.03529412, 0.07450981],
         [0.27450982, 0.1764706 , 0.16078432]],

        [[0.8039216 , 0.72156864, 0.5803922 ],
         [0.7647059 , 0.6901961 , 0.5176471 ],
         [0.7529412 , 0.6745098 , 0.49019608],
         ...,
         [0.23529412, 0.01568628, 0.06666667],
         [0.25490198, 0.13333334, 0.11764706],
         [0.3764706 , 0.39607844, 0.29411766]],

        ...,

        [[0.29803923, 0.34117648,

In [5]:
fb.utils.accuracy(fmodel, images, labels)
# -> 0.9375 (depends on the model!)

0.875

In [6]:
attack = fb.attacks.FGSM()

In [7]:
raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.03)

# FGSM 2

In [13]:
cd "C:\Users\Francisco Duran\Desktop\upc\project_notebooks"

C:\Users\Francisco Duran\Desktop\upc\project_notebooks


In [20]:
from simplecnn import SimpleCNN
from fgsm import generate_image_adversary
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2


In [15]:
# Load the MNIST dataset and scale the pixel values to the range [0, 1]
print("[INFO] Loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

[INFO] Loading MNIST dataset...


In [18]:
# Add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# One hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

In [28]:
trainX

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [27]:
testY.shape

(10000, 10)

In [21]:
# Initialize the optimizer and the model
print("[INFO] Compiling the model...")
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] Compiling the model...


In [22]:
# Train the simple CNN on the MNIST dataset
print("[INFO] Training the network...")
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=10, verbose=1)

# Make predictions on the testing set for the model trained on non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("[INFO] Loss: {:.4f}, Accuracy: {:.4f}".format(loss, acc))

[INFO] Training the network...
Epoch 1/10
938/938 [==============================] - 22s 22ms/step - loss: 0.1954 - accuracy: 0.9420 - val_loss: 0.0661 - val_accuracy: 0.9796
Epoch 2/10
938/938 [==============================] - 20s 22ms/step - loss: 0.0784 - accuracy: 0.9761 - val_loss: 0.0415 - val_accuracy: 0.9856
Epoch 3/10
938/938 [==============================] - 21s 22ms/step - loss: 0.0578 - accuracy: 0.9823 - val_loss: 0.0385 - val_accuracy: 0.9870
Epoch 4/10
938/938 [==============================] - 20s 22ms/step - loss: 0.0489 - accuracy: 0.9850 - val_loss: 0.0372 - val_accuracy: 0.9876
Epoch 5/10
938/938 [==============================] - 21s 22ms/step - loss: 0.0393 - accuracy: 0.9876 - val_loss: 0.0391 - val_accuracy: 0.9873
Epoch 6/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0347 - accuracy: 0.9890 - val_loss: 0.0372 - val_accuracy: 0.9884
Epoch 7/10
938/938 [==============================] - 20s 22ms/step - loss: 0.0294 - accuracy: 0.9906 - v

In [ ]:
# Loop over a sample of the testing images
for i in np.random.choice(np.arange(0, len(testX)), size=(10,)):
    # Grab the current image and label
    image = testX[i]
    label = testY[i]
    # Generate an image adversary for the current image and make a prediction on the adversary image
    adversary = generate_image_adversary(model, image.reshape(1, 28, 28, 1), label, eps=0.1)
    pred = model.predict(adversary)
    # Scale both the original image and the adversary image to the range [0, 255]
    # and convert them to unsigned 8-bit integers
    adversary = adversary.reshape((28, 28)) * 255
    adversary = np.clip(adversary, 0, 255).astype("uint8")
    image = image.reshape((28, 28)) * 255
    image = image.astype("uint8")
    # Convert the image and adversarial image from grayscale to three channel (in order to draw on the image)
    image = np.dstack([image] * 3)
    adversary = np.dstack([adversary] * 3)
    # Resize the images in order to visualize them later
    image = cv2.resize(image, (96, 96))
    adversary = cv2.resize(adversary, (96, 96))
    # Determine the predicted label for both the original image and the adversarial image
    imagePred = label.argmax()
    adversaryPred = pred[0].argmax()
    color = (0, 255, 0)
    # If the image prediction does not match with the adversarial prediction then update the color
    if imagePred != adversaryPred:
        color = (0, 0, 255)
    # Draw the predictions on the respective output images
    cv2.putText(image, str(imagePred), (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2)
    cv2.putText(adversary, str(adversaryPred), (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)
    # Stack the two images horizontally and then show the original image and its adversary
    output = np.hstack([image, adversary])
    cv2.imshow("FGSM Adversarial Images", output)
    cv2.waitKey(0)
